<a href="https://colab.research.google.com/github/nebyu08/data_sc/blob/main/advanced_sentimental_analysis_pschology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers
!pip install torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.5 MB/s eta 0:00:00


# define the model

In [15]:
import torch
import transformers

# Load the model and tokenizer
model = transformers.AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")
tokenizer = transformers.AutoTokenizer.from_pretrained("facebook/bart-large-mnli")


In [27]:
def classify(text,model):
  # Define the candidate labels
  labels = ["depression", "anxiety", "bipolar disorder", "schizophrenia", "PTSD", "OCD", "ADHD", "autism", "eating disorder", "personality disorder", "phobia"]
  # Split the text into segments
  segments = split_text(text)
  # Initialize an empty list for logits
  logits_list = []
  #move device to GPU
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device) # Move the model to the device
  # Loop through the segments
  for segment in segments:
    # Encode the segment and the labels
    inputs = tokenizer([segment] + labels, padding=True, return_tensors="pt")
    # Get the input ids and attention mask
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    # Move the input ids and attention mask to the device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    #Get the model outputs for each segment
    outputs = model(
      input_ids,
      attention_mask=attention_mask,
    )
    # Get the logits for each segment and append them to the logits list
    logits = outputs.logits
    logits_list.append(logits)
  # Average the logits across the segments
  avg_logits = torch.mean(torch.stack(logits_list), dim=0)
  # Get the entailment scores for each label from the averaged logits
  scores = avg_logits[:, :]
  # Sort the scores and get the indices
  sorted_scores, indices = torch.sort(scores, descending=True)
  # Get the sorted labels and scores
  sorted_labels = [labels[i.item()] for i in indices.flatten()]
  sorted_scores = sorted_scores.tolist()
  # Return the results as a dictionary
  return {
    "sequence": text,
    "labels": sorted_labels,
    "scores": sorted_scores
  }

In [28]:
# Define a text to classify
text = "I feel restless and irritable most of the time. I have trouble concentrating and making decisions. I often have mood swings and act impulsively."

In [29]:
print(split_text(text))

['I feel restless and irritable most of the time.', ' I have trouble concentrating and making decisions.', ' I often have mood swings and act impulsively.']


In [30]:
# Call the classify function using sliding window approach
output = classify(text,model)

In [31]:
# Print the output
print(output)

{'sequence': 'I feel restless and irritable most of the time. I have trouble concentrating and making decisions. I often have mood swings and act impulsively.', 'labels': ['anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression', 'anxiety', 'bipolar disorder', 'depression'], 'scores': [[4.015264511108398, -1.198138952255249, -2.654818534851074], [1.1328034400939941, 1.0545679330825806, -1.9834131002426147], [1.0575324296951294, 0.9243408441543579, -2.2091121673583984], [1.6687244176864624, 0.6978153586387634, -2.4287972450256348], [1.1087653636932373, 1.005